In [2]:
import numpy as np
import pdb
from matplotlib import pyplot as plt

# Parameters
k = 386 # W/m/K
alpha = 1E6
r1 = 1E-3 # m
r2 = 4E-3 # m
T1 = 353.15 # K (80C)
T2 = 293.15 # K (20C)
m  = 5   # Number of elements 

# Select Method
FEMApprox = True # True if FEM Approx else Galerkin Approx

# Create empty matrices
n = m+1 # Amount of dofs
K = np.mat(np.zeros((n,n)))
F = np.mat(np.zeros((n,1)))

In [5]:
# FEM approximation    
# Dof locations
r = np.linspace(r1, r2, n)

# Define basis function
def N(A,ni,r): # A: basis function, ni: node, r: dof locations
    NA = 0
    if r[A-1] <= r[ni] and r[ni] <= r[A]:
        NA += m*(r[ni] - r[A-1])/(r2 - r1)         
    elif r[A] <= r[ni] and r[ni] <= r[A+1]:
        NA += m*(r[A+1] - r[ni])/(r2 - r1)
    return NA

# Stiffness matrix
def Kij(a,b):
    if a==0 and b==0:
        Kij = k * (1/((r[1] - r[0])**2)) * (r[1]**2 - r[0]**2)/2
    elif a==m and b==m:
        Kij = k * (1/((r[m] - r[m-1])**2)) * (r[m]**2 - r[m-1]**2)/2
    elif a==b:
        Kij = k * (1/((r[a] - r[a-1])**2)) * (r[a]**2 - r[a-1]**2)/2 \
            + k * (1/((r[a+1] - r[a])**2)) * (r[a+1]**2 - r[a]**2)/2 
    elif b==a-1:
        Kij = -k * (1/((r[a] - r[a-1])**2)) * (r[a]**2 - r[a-1]**2)/2
    elif b==a+1:
        Kij = -k * (1/((r[a+1] - r[a])**2)) * (r[a+1]**2 - r[a]**2)/2
    else:
        Kij = 0
    Kij += alpha*r2*N(a,n-1,r)*N(b,n-1,r)
    return Kij
    
# Fill matrix and vector
for i in range(n):
    for j in range(n):
        K[i,j]  = Kij(i,j)
    F[i] = alpha*r2*N(i,n-1,r)*T2
print(K)
print(F)

lhs = K
rhs = F

"""
# Apply constraints
C = ... # TODO

# Lift function
qh_hat = ...# TODO

# Matrix to solve: lhs and rhs
lhs = ...
rhs = ...
"""

# Solve
Th_hat = lhs.I*rhs

# Compute fields
Th = np.vstack([np.array([T1]),Th_hat])
r = np.linspace(r1, r2, len(Th))

[[  836.33333333  -836.33333333     0.             0.
      0.             0.        ]
 [ -836.33333333  2058.66666667 -1222.33333333     0.
      0.             0.        ]
 [    0.         -1222.33333333  2830.66666667 -1608.33333333
      0.             0.        ]
 [    0.             0.         -1608.33333333  3602.66666667
  -1994.33333333     0.        ]
 [    0.             0.             0.         -1994.33333333
   4374.66666667 -2380.33333333]
 [    0.             0.             0.             0.
  -2380.33333333  6380.33333333]]


In [ ]:
 # Fill matrix and vector
for i in range(1,n+1):
    for j in range(1,n+1):
        K[i-1,j-1] = k*i*j*(r2-r1)**(i+j-1)*(r1+(i+j-1)*r2)/((i+j-1)*(i+j))
        K[i-1,j-1] += alpha*r2*(r2-r1)**j*(r2-r1)**i
    F[i-1] = alpha*r2*(T2-T1)*(r2-r1)**i
print(K)
print(F)

# Solve
Th_hat = K.I*F

# Compute fields
r = np.linspace(r1, r2, 100)
Th = T1+sum( [ float(Th_hat[i-1])*(r-r1)**i for i in range(1,n+1)] )

In [ ]:

# Exact solution
ra = np.linspace(r1, r2, 10)
C1 = - (T1-T2)/( k/(alpha*r2)+np.log(r2/r1) )
T= C1*np.log(ra/r1)+T1

# Plot data
plt.plot(r,Th,'r-',label='$T^h$') # Approximation
plt.plot(ra,T,'k:',label='$T$') # Exact temperature
plt.plot([0,r1],[T1,T1],'k:') # Exact temperature inside
plt.plot([r2,r2+0.5*r1],[T2,T2],'k:') # Exact temperature outside

# Style graph
plt.xlabel('$r$ [m]')
plt.ylabel('$T$ [K]')
plt.legend()
plt.show()